In [4]:
## 새로운 이미지 추가시 유사 이미지 찾기

# find_images_VGG16(img_name, top_num)
# img_name: 새로운 이미지 파일명

def find_images_VGG16(img_name, top_num):
    from PIL import Image
    import os
    import six.moves.cPickle as Pickle

    from keras.applications.vgg16 import VGG16
    from keras.preprocessing import image
    from keras.applications.vgg16 import preprocess_input
    from keras.models import Model
    
    import numpy as np
    from sklearn.metrics.pairwise import euclidean_distances

    ## VGG16
    vgg_model = VGG16(weights='imagenet')
    model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('fc2').output) 

    img_path = '../ex/' + img_name + '.png'  # 새로운 이미지 저장된 경로 지정

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    ex_feature = model.predict(x)


    ## 기존 이미지(lookbook -cloth_table)의 feature vector 불러오기   
    with open('vgg16_features_4096.pkl', 'rb') as vgg16_features_4096:
        vgg16_features_4096 = Pickle.load(vgg16_features_4096)


    ## 이미지간 거리 계산(euclidean distance)
    dist = euclidean_distances(ex_feature, vgg16_features_4096)
    dist = np.squeeze(dist)


    ## 기존 이미지 불러오기
    with open('cloth_table.pkl', 'rb') as cloth_table:
        cloth_data = Pickle.load(cloth_table)
 

    ## 유사 이미지 선택
    top_ind = np.argsort(dist)[:top_num]
    result_img = []
    result = Image.open(img_path) 
    result_img.append(np.array(result))
    
    for ind in top_ind:
        img_path2 = '../data/' + cloth_data[ind]
        result2 = Image.open(img_path2)
        result_img.append(np.array(result2))
    
    return result_img  



def plot(samples, top_num):  # samples: numpy 값으로 넣어야 한다!
    import matplotlib.gridspec as gridspec
    import matplotlib.pylab as plt
    import numpy as np

    fig = plt.figure(figsize=(20,40))
    gs = gridspec.GridSpec(1,(top_num+1))
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis("off")  # 축 제거
        sample = sample.astype(np.uint8)
        a = plt.imshow(sample)
    plt.show()    



## 함수 실행
plot(find_images_VGG16("ex2", 5),5)    
find_images_VGG16("ex2",5) 